In [1]:
import networkx as nx
import random
import heapq
import matplotlib.pyplot as plt
import numpy as np
import time
from tqdm import tqdm
import pandas as pd

In [2]:

def average_edge_probability(G):
    """ Tính xác suất trung bình của các cạnh trong đồ thị G. """
    total_probability = sum(nx.get_edge_attributes(G, 'probability').values())
    num_edges = G.number_of_edges()
    return total_probability / num_edges if num_edges > 0 else 0

def calculate_edge_density_v2(G):
    """Tính mật độ cạnh kỳ vọng của đồ thị G. τ"""
    num_vertices = len(G.nodes())
    num_possible_edges = num_vertices * (num_vertices - 1) if num_vertices > 1 else 1
    sum_weighted_probabilities = sum(G[u][v]['weight'] * G[u][v]['probability'] for u, v in G.edges())
    return sum_weighted_probabilities / num_possible_edges

def expected_degree_out(G, v):
    return sum(G[v][w]['weight'] *G[v][w]['probability'] for w in G.successors(v) if w in G)

def expected_degree_in(G, v):
    return sum(G[u][v]['weight'] *G[u][v]['probability'] for u in G.predecessors(v) if u in G)

# Hàm tính mật độ mong đợi
def expected_density_v3(G, S, T):
    if G.number_of_edges() == 0:
        return 0
    probability_sum = sum(G[u][v]['weight'] * G[u][v]['probability'] for u, v in G.edges() if u in S and v in T)
    return probability_sum / (len(S) * len(T))**0.5



# Hàm tính xác suất trung bình của các cạnh trong đồ thị
def average_edge_probability(G):
    total_probability = sum(nx.get_edge_attributes(G, 'probability').values())
    num_edges = G.number_of_edges()
    return total_probability / num_edges if num_edges > 0 else 0

def weighted_average_edge_probability(G):
    total_weighted_probability = sum(G[u][v]['weight'] * G[u][v]['probability'] for u, v in G.edges())  
    num_edges = G.number_of_edges()
    return total_weighted_probability / num_edges if num_edges > 0 else 0


def greedy_approximation(G, c, S, T):
    H = G.copy()
    best_graph = H.copy()
    S_temp = S.copy()
    T_temp = T.copy()    
    highest_density = expected_density_v3(H, S_temp, T_temp)
    
    num_iterations = len(S_temp) + len(T_temp)  # Tổng số đỉnh ban đầu trong cả hai tập
    with tqdm(total=num_iterations, desc="Processing", unit="node") as pbar:
        while S_temp and T_temp:
            i_min = min(S_temp, key=lambda i: expected_degree_out(H, i))
            d_S = expected_degree_out(H, i_min)

            j_min = min(T_temp, key=lambda j: expected_degree_in(H, j))
            d_T = expected_degree_in(H, j_min)
            
            if c * d_S <= d_T / c:
                # Loại bỏ i_min khỏi S_temp, hưng giữ lại trong T_temp nếu có
                S_temp.remove(i_min)
                if i_min not in T_temp:  # Nếu i_min không có trong T_temp, loại nó khỏi đồ thị
                    H.remove_node(i_min)
                
            else:
                # Loại bỏ j_min khỏi T_temp, nhưng giữ lại trong S_temp nếu có
                T_temp.remove(j_min)
                if j_min not in S_temp:  # Nếu j_min không có trong S_temp, loại nó khỏi đồ thị
                    H.remove_node(j_min)
                
            
            current_density = expected_density_v3(H, S_temp, T_temp)
            
            # Cập nhật lại S_temp và T_temp với những đỉnh hiện có trong H
            S_temp = {u for u in S_temp if u in H}
            T_temp = {v for v in T_temp if v in H}
            
            if current_density > highest_density:
                highest_density = current_density
                best_graph = H.copy()  # Lưu bản sao của H với mật độ cao nhất

            if not S_temp or not T_temp:
                break
            
            pbar.update(1)  # Cập nhật tiến trình mỗi khi một đỉnh được loại bỏ
    
    return best_graph


In [3]:
random.seed(10)
# Đọc dữ liệu từ tệp TXT
file_path = '/teamspace/uploads/data/300267.protein.links.full.v12.0.txt'
data = pd.read_csv(file_path, sep=' ')

# Khởi tạo đồ thị có hướng
G = nx.DiGraph()

# Tìm trọng số lớn nhất trong cột 'experiments'
max_experiment_weight = data['experiments_transferred'].max()

# Thêm các cạnh với trọng số và xác suất
for _, row in data.iterrows():
    protein1 = row['protein1']
    protein2 = row['protein2']
    experiment_weight = row['experiments_transferred']

    # Tính tỷ lệ trọng số so với trọng số lớn nhất
    if max_experiment_weight > 0:
        normalized_weight = experiment_weight / max_experiment_weight
    else:
        normalized_weight = 0  # Tránh chia cho 0 trong trường hợp không có trọng số nào

    # Giả sử xác suất được tính dựa trên combined_score
    combined_score = row['combined_score']
    probability = combined_score / 1000.0  # Điều chỉnh hệ số nếu cần thiết

    # Định hướng cạnh dựa trên một quy tắc ngẫu nhiên
    if random.choice([True, False]):
        G.add_edge(protein1, protein2, probability=probability, weight=normalized_weight)
        
        
S = set()
T = set()
for u, v in G.edges():
    S.add(u)
    T.add(v)
# print("tập S có:",len(S))
# print("tập T có:",len(T))

print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())


# Lấy các giá trị xác suất từ các cạnh
probabilities = [G[u][v]['probability'] for u, v in G.edges()]
weights = [G[u][v]['weight'] for u, v in G.edges()]
# Tính giá trị trung bình
mean_probability = np.mean(probabilities)
mean_weight = np.mean(weights)


# Tính độ lệch chuẩn
std_deviation = np.std(probabilities)
std_weight = np.std(weights)

print("Mật độ cạnh kì vọng của đồ thị là: ", calculate_edge_density_v2(G))
print("Độ lệch chuẩn xác xuất :", std_deviation)
print("Độ lệch chuẩn trọng số :",std_weight)
print("Giá trị cạnh trung bình của đồ thị protein579138:", mean_probability)
print("Trọng số cạnh trung bình của đồ thị là:",mean_weight)
print("Trung bình trọng số xác xuất :",weighted_average_edge_probability(G))



Number of nodes: 4127
Number of edges: 299411
Mật độ cạnh kì vọng của đồ thị là:  0.0003527837482851522
Độ lệch chuẩn xác xuất : 0.17910467648035774
Độ lệch chuẩn trọng số : 0.12597629340607686
Giá trị cạnh trung bình của đồ thị protein579138: 0.28025207824695825
Trọng số cạnh trung bình của đồ thị là: 0.0338805050902612
Trung bình trọng số xác xuất : 0.020063399044681286


In [ ]:
c = len(S) / len(T)
uds_wiki_vote = greedy_approximation(G, c, S, T)

S = set()
T = set()
for u, v in uds_wiki_vote.edges():
    S.add(u)
    T.add(v)
# Lấy các giá trị xác suất từ các cạnh
probabilities = [uds_wiki_vote[u][v]['probability'] for u, v in uds_wiki_vote.edges()]
weights = [uds_wiki_vote[u][v]['weight'] for u, v in uds_wiki_vote.edges()]

# Tính giá trị trung bình
mean_probability = np.mean(probabilities)
mean_weight = np.mean(weights)


# Tính độ lệch chuẩn
std_deviation = np.std(probabilities)
std_weight = np.std(weights)

print("Mật độ kì vọng của đồ thị con do thuật toán obs sinh ra là:", expected_density_v3(uds_wiki_vote, S, T))
print("Mật độ cạnh kì vọng của đồ thị con được tạo ra bởi thuật toán uds là:",calculate_edge_density_v2(uds_wiki_vote))
print("Xác xuất cạnh trung bình của đồ thị con được tạo ra bởi thuật toán uds là:",average_edge_probability(uds_wiki_vote))
print("Độ lệch chuẩn xác xuất:", std_deviation)
print("Độ lệch chuẩn trọng số:", std_weight)
print("Trọng số cạnh trung bình của thuật toán obs là:",mean_weight)
print("số đỉnh của uds_wiki_vote:", len(uds_wiki_vote.nodes))
print("Số cạnh của uds_wiki_vote:", len(uds_wiki_vote.edges))
print("Trung bình trọng số xác xuất :",weighted_average_edge_probability(uds_wiki_vote))



Processing:  28%|██▊       | 2255/8055 [1:03:05<2:13:18,  1.38s/node]